<a href="https://colab.research.google.com/github/sujal029/pyspark-etl-pipeline/blob/main/Copy_of_DE_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O spark.tgz https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz


--2025-06-19 10:46:35--  https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388341449 (370M) [application/x-gzip]
Saving to: ‘spark.tgz’

spark.tgz           100%[===================>] 370.35M  9.28MB/s    in 29s     

2025-06-19 10:47:04 (12.8 MB/s) - ‘spark.tgz’ saved [388341449/388341449]



In [ ]:
!tar -xvzf spark.tgz


spark-3.4.1-bin-hadoop3/
spark-3.4.1-bin-hadoop3/R/
spark-3.4.1-bin-hadoop3/R/lib/
spark-3.4.1-bin-hadoop3/R/lib/sparkr.zip
spark-3.4.1-bin-hadoop3/R/lib/SparkR/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/R.css
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/00Index.html
spark-3.4.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/aliases.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/AnIndex
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdx
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdb
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/paths.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/worker.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/daemon.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.4.1-bin-hadoop3/R/lib/S

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ZelestraApp").getOrCreate()


In [ ]:
import pandas as pd

df1 = pd.DataFrame({
    "datetime": pd.date_range("2024-08-01", periods=10, freq="D"),
    "value": range(10)
})
df1.to_parquet("/content/sensor_data_1.parquet", index=False)

df2 = pd.DataFrame({
    "datetime": pd.date_range("2024-08-05", periods=10, freq="D"),
    "value": range(10, 20)
})
df2.to_parquet("/content/sensor_data_2.parquet", index=False)

print("✅ Dummy Parquet Files Created.")


✅ Dummy Parquet Files Created.


In [ ]:
import pandas as pd

df1 = pd.DataFrame({
    "datetime": pd.date_range("2024-08-01", periods=10, freq="D"),
    "value": range(10)
})
df1.to_parquet("/content/sensor_data_1.parquet", index=False, engine="pyarrow")

df2 = pd.DataFrame({
    "datetime": pd.date_range("2024-08-05", periods=10, freq="D"),
    "value": range(10, 20)
})
df2.to_parquet("/content/sensor_data_2.parquet", index=False, engine="pyarrow")

print("✅ Dummy parquet files re-saved correctly.")


✅ Dummy parquet files re-saved correctly.


In [ ]:
import os

print("sensor_data_1.parquet:", os.path.exists("/content/sensor_data_1.parquet"))
print("sensor_data_2.parquet:", os.path.exists("/content/sensor_data_2.parquet"))


sensor_data_1.parquet: True
sensor_data_2.parquet: True


In [ ]:
from pyspark.sql.functions import col, to_timestamp
from datetime import datetime, timedelta

files = {
    'sensor_data_1.parquet': 'sensor_1',
    'sensor_data_2.parquet': 'sensor_2'
}
actual_data = {}
last_dates = {}

for filename, var_name in files.items():
    print(f"Reading {filename}")
    df = spark.read.parquet(f"/content/{filename}")
    df = df.withColumn("datetime", to_timestamp(col("datetime")))
    actual_data[filename] = df
    last_dates[var_name] = df.agg({"datetime": "max"}).collect()[0][0]

# Use last 30 days
one_month_ago = datetime.now() - timedelta(days=30)
filtered = {k: v for k, v in last_dates.items() if v and v >= one_month_ago}
min_date = min(filtered.values()) if filtered else datetime(2024, 8, 1)

print("📅 Min valid date for processing:", min_date)


Reading sensor_data_1.parquet


Py4JJavaError: An error occurred while calling o30.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (afba865e05fe executor driver): org.apache.spark.sql.AnalysisException: Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false)).
	at org.apache.spark.sql.errors.QueryCompilationErrors$.illegalParquetTypeError(QueryCompilationErrors.scala:1762)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.illegalType$1(ParquetSchemaConverter.scala:206)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.$anonfun$convertPrimitiveField$2(ParquetSchemaConverter.scala:283)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSparkSchemaConverter.convertPrimitiveField(ParquetSchemaConverter.scala:224)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetToSpa

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

df1 = pd.DataFrame({
    "datetime": pd.date_range("2024-08-01", periods=10, freq="D"),
    "value": range(10)
})
df2 = pd.DataFrame({
    "datetime": pd.date_range("2024-08-05", periods=10, freq="D"),
    "value": range(10, 20)
})

# Force timestamp in ms precision (Spark-compatible)
def save_parquet_spark_compatible(df, path):
    table = pa.Table.from_pandas(df, schema=None, preserve_index=False)
    table = table.cast(pa.schema([
        ("datetime", pa.timestamp('ms')),
        ("value", pa.int64())
    ]))
    pq.write_table(table, path)

save_parquet_spark_compatible(df1, "/content/sensor_data_1.parquet")
save_parquet_spark_compatible(df2, "/content/sensor_data_2.parquet")

print("✅ Saved both files with Spark-compatible datetime format.")


✅ Saved both files with Spark-compatible datetime format.


In [ ]:
df = spark.read.parquet("/content/sensor_data_1.parquet")
df.show()


+-------------------+-----+
|           datetime|value|
+-------------------+-----+
|2024-08-01 00:00:00|    0|
|2024-08-02 00:00:00|    1|
|2024-08-03 00:00:00|    2|
|2024-08-04 00:00:00|    3|
|2024-08-05 00:00:00|    4|
|2024-08-06 00:00:00|    5|
|2024-08-07 00:00:00|    6|
|2024-08-08 00:00:00|    7|
|2024-08-09 00:00:00|    8|
|2024-08-10 00:00:00|    9|
+-------------------+-----+



In [ ]:
tags_data = [
    {"id": 101, "tagpath": "sensor/module/temp_1"},
    {"id": 102, "tagpath": "sensor/module/temp_2"},
    {"id": 103, "tagpath": "sensor/module/temp_3"},
]
tags_df = spark.createDataFrame(tags_data)
tags_df.show()


+---+--------------------+
| id|             tagpath|
+---+--------------------+
|101|sensor/module/temp_1|
|102|sensor/module/temp_2|
|103|sensor/module/temp_3|
+---+--------------------+



In [ ]:
from pyspark.sql import Row
from datetime import datetime, timedelta

# Simulate 3 sensor readings from 3 tags over 10 days
base_date = datetime(2024, 8, 1)
data_rows = []
for i in range(10):
    for tag_id in [101, 102, 103]:
        data_rows.append(Row(
            tagid=tag_id,
            t_stamp=int((base_date + timedelta(days=i)).timestamp() * 1000),  # ms
            value=tag_id * 100 + i,
            dataintegrity=1
        ))

sensor_df = spark.createDataFrame(data_rows)
sensor_df.show(5)


+-----+-------------+-----+-------------+
|tagid|      t_stamp|value|dataintegrity|
+-----+-------------+-----+-------------+
|  101|1722470400000|10100|            1|
|  102|1722470400000|10200|            1|
|  103|1722470400000|10300|            1|
|  101|1722556800000|10101|            1|
|  102|1722556800000|10201|            1|
+-----+-------------+-----+-------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import from_unixtime, col

# Join tags
merged_df = sensor_df.join(tags_df, sensor_df.tagid == tags_df.id, "left")

# Convert timestamp and filter bad data
cleaned = (
    merged_df.filter(col("dataintegrity") != 0)
    .withColumn("datetime", from_unixtime(col("t_stamp") / 1000))
    .select("datetime", "tagpath", "value")
)

cleaned.show(5)


+-------------------+--------------------+-----+
|           datetime|             tagpath|value|
+-------------------+--------------------+-----+
|2024-08-01 00:00:00|sensor/module/temp_3|10300|
|2024-08-02 00:00:00|sensor/module/temp_3|10301|
|2024-08-01 00:00:00|sensor/module/temp_1|10100|
|2024-08-02 00:00:00|sensor/module/temp_1|10101|
|2024-08-01 00:00:00|sensor/module/temp_2|10200|
+-------------------+--------------------+-----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import regexp_replace

unique_tags = [row['tagpath'] for row in cleaned.select("tagpath").distinct().collect()]

for tag in unique_tags:
    tag_df = cleaned.filter(col("tagpath") == tag).dropna()
    if tag_df.count() == 0:
        continue

    col_name = tag.replace("/", "_")
    tag_df = tag_df.select("datetime", col("value").alias(col_name))

    save_path = f"/content/{col_name}.parquet"
    tag_df.write.mode("overwrite").parquet(save_path)
    print(f"✅ Saved file: {save_path}")


✅ Saved file: /content/sensor_module_temp_2.parquet
✅ Saved file: /content/sensor_module_temp_1.parquet
✅ Saved file: /content/sensor_module_temp_3.parquet


In [ ]:
!zip -r DE_Assignment_Output.zip /content/sensor_module_temp_*.parquet


  adding: content/sensor_module_temp_1.parquet/ (stored 0%)
  adding: content/sensor_module_temp_1.parquet/part-00000-5a1a8360-1fde-4419-af3f-4099312f9bbc-c000.snappy.parquet (deflated 41%)
  adding: content/sensor_module_temp_1.parquet/.part-00000-5a1a8360-1fde-4419-af3f-4099312f9bbc-c000.snappy.parquet.crc (stored 0%)
  adding: content/sensor_module_temp_1.parquet/_SUCCESS (stored 0%)
  adding: content/sensor_module_temp_1.parquet/._SUCCESS.crc (stored 0%)
  adding: content/sensor_module_temp_2.parquet/ (stored 0%)
  adding: content/sensor_module_temp_2.parquet/.part-00000-e68d4d1d-de72-42be-8f01-e13230405c50-c000.snappy.parquet.crc (stored 0%)
  adding: content/sensor_module_temp_2.parquet/part-00000-e68d4d1d-de72-42be-8f01-e13230405c50-c000.snappy.parquet (deflated 40%)
  adding: content/sensor_module_temp_2.parquet/_SUCCESS (stored 0%)
  adding: content/sensor_module_temp_2.parquet/._SUCCESS.crc (stored 0%)
  adding: content/sensor_module_temp_3.parquet/ (stored 0%)
  adding: cont

In [ ]:
from google.colab import files

# Save the current notebook
!jupyter nbconvert --to notebook --output DE_Assignment.ipynb /content/*.ipynb


[NbConvertApp] WARNING | pattern '/content/*.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    